In [1]:
# Instala versiones compatibles para evitar incompatibilidades de C headers
!pip install numpy==1.23.5 mo-gymnasium --force-reinstall --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.6/479.6 kB 217.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 242.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 233.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 262.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 167.8 MB/s eta 0:00:00
  Attempting uninstall: farama-notifications
    Found existing installation: Farama-Notifications 0.0.4
    Uninstalling Farama-Notifications-0.0.4:
      Successfully uninstalled Farama-Notifications-0.0.4
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.2
    Uninstalling typing_extensions-4.13.2:
      Successfully uninstalled typing_extensions-4.13.2
  Attempting uninstall: pygame


In [2]:
# 1. Instalar SWIG (requerido por Box2D)
!apt-get install -y swig

# 2. Instalar Box2D y entornos de física para gymnasium
!pip install "gymnasium[box2d]" mo-gymnasium numpy==1.23.5 --force-reinstall --no-cache-dir


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 0s (4,505 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 126315 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [1]:
import mo_gymnasium as mo_gym

env = mo_gym.make("mo-lunar-lander-v3")
obs, _ = env.reset()
print("Recompensa de ejemplo:", env.step(env.action_space.sample())[1])



Recompensa de ejemplo: [ 0.        -2.4598498 -1.        -0.       ]


/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [2]:
# prueba del entorno
import numpy as np

obs, _ = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    obs, reward_vector, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    print("Recompensa vectorial:", reward_vector)


Recompensa vectorial: [ 0.         1.3467916 -0.        -1.       ]
Recompensa vectorial: [ 0.        -3.1804328 -1.        -0.       ]
Recompensa vectorial: [ 0.         2.4186897 -0.        -1.       ]
Recompensa vectorial: [ 0.         2.0832074 -0.        -0.       ]
Recompensa vectorial: [ 0.         2.0190084 -0.        -0.       ]
Recompensa vectorial: [ 0.       -3.395525 -1.       -0.      ]
Recompensa vectorial: [ 0.         2.0583832 -0.        -0.       ]
Recompensa vectorial: [ 0.         1.9427717 -0.        -0.       ]
Recompensa vectorial: [ 0.        -0.4122251 -1.        -0.       ]
Recompensa vectorial: [ 0.        1.522646 -0.       -0.      ]
Recompensa vectorial: [ 0.         2.0045798 -0.        -1.       ]
Recompensa vectorial: [ 0.          0.00644553 -0.         -0.        ]
Recompensa vectorial: [ 0.         -0.02527957 -0.         -1.        ]
Recompensa vectorial: [ 0.        -0.4067387 -1.        -0.       ]
Recompensa vectorial: [ 0.         3.1113844 -1.

In [10]:
import numpy as np
import random

class ParetoQLearning:
    def __init__(self, env, gamma=0.9, alpha=0.1):
        self.env = env
        self.gamma = gamma
        self.alpha = alpha

        self.Q = {}  # Diccionario: (s,a) → lista de vectores no dominados
        self.reward_dim = env.unwrapped.reward_space.shape[0]
        self.n_actions = env.action_space.n

    def is_dominated(self, v1, v2):
        return all(v1_i <= v2_i for v1_i, v2_i in zip(v1, v2)) and any(v1_i < v2_i for v1_i, v2_i in zip(v1, v2))

    def remove_dominated(self, vectors):
        nd = []
        for v in vectors:
            if not any(self.is_dominated(v, u) for u in vectors if not np.allclose(v, u)):
                nd.append(v)
        return nd

        def select_action(self, s, weights, epsilon=0.1):
            if random.random() < epsilon:
              return self.env.action_space.sample()

            best_score = -np.inf
            best_action = random.choice(range(self.n_actions))
            for a in range(self.n_actions):
                for q in self.Q.get((s, a), []):
                    score = np.dot(weights, q)
                    if score > best_score:
                        best_score = score
                        best_action = a
        return best_action

    def update(self, s, a, r_vec, s_next):
        key = (s, a)

        # Construir sucesores
        successors = []
        for a_prime in range(self.n_actions):
            for q in self.Q.get((s_next, a_prime), []):
                successors.append(q)

        if successors:
            # Suma recompensa inmediata + γ * q' para cada q' no dominado
            candidates = [r_vec + self.gamma * np.array(q) for q in self.remove_dominated(successors)]
        else:
            # No hay Q futuros (ej: estado terminal)
            candidates = [r_vec]

        # Actualizar el conjunto Q(s,a)
        self.Q[key] = self.remove_dominated(self.Q.get(key, []) + candidates)

    def train(self, episodes, weights):
        for ep in range(1, episodes + 1):
            obs, _ = self.env.reset()
            s = tuple(obs.round(2))  # Convertimos el estado continuo en clave hashable
            done = False
            total_reward = np.zeros(self.reward_dim)
            steps = 0

            print(f"\nEpisodio {ep} iniciado")

            while not done:
                a = self.select_action(s, weights, epsilon=0.1)
                obs_next, r_vec, terminated, truncated, _ = self.env.step(a)
                done = terminated or truncated
                s_next = tuple(obs_next.round(2))

                self.update(s, a, np.array(r_vec), s_next)

                total_reward += r_vec
                s = s_next
                steps += 1

            print(f"Episodio {ep} terminado en {steps} pasos")
            print(f"Recompensa acumulada: {total_reward}")




IndentationError: expected an indented block after function definition on line 24 (<ipython-input-10-905c8d6c8a3c>, line 25)

In [7]:
env = mo_gym.make("mo-lunar-lander-v3")
agent = ParetoQLearning(env)

# Preferencia en pesos: [aterrizaje, trayectoria, motor principal, motor lateral]
weights = np.array([0.5, 0.3, 0.1, 0.1])

agent.train(episodes=500, weights=weights)  # prueba con pocos episodios para empezar



Episodio 1 iniciado
Episodio 1 terminado en 73 pasos
Recompensa acumulada: [-100.          130.87047167  -29.          -30.        ]

Episodio 2 iniciado
Episodio 2 terminado en 90 pasos
Recompensa acumulada: [-100.           92.04050981  -27.          -40.        ]

Episodio 3 iniciado
Episodio 3 terminado en 96 pasos
Recompensa acumulada: [-100.         -103.92844937  -19.          -53.        ]

Episodio 4 iniciado
Episodio 4 terminado en 102 pasos
Recompensa acumulada: [-100.           37.83170594  -34.          -46.        ]

Episodio 5 iniciado
Episodio 5 terminado en 110 pasos
Recompensa acumulada: [-100.           15.41353609  -29.          -56.        ]

Episodio 6 iniciado
Episodio 6 terminado en 81 pasos
Recompensa acumulada: [-100.          130.58058426  -20.          -39.        ]

Episodio 7 iniciado
Episodio 7 terminado en 99 pasos
Recompensa acumulada: [-100.        -262.6394404  -23.         -57.       ]

Episodio 8 iniciado
Episodio 8 terminado en 98 pasos
Recompensa